In [1]:
# указываем путь к рабочей папке, где будут храниться фото и аудиозаписи, а также база данных
# там же храним каскады Хааара
# указываем токен бота

# ! вставьте сюда путь к рабочей папке (туда же надо скопировать файл с каскадом хаара)
path = r'/home/mikhail/DSPtest'

# ! вставьте сюда токен своего бота
bot_token = '<>'

# ! работает только при включённом vpn

In [2]:
# подключаем библиотеки
import cv2
import sqlite3
from pydub import AudioSegment
import json
import telebot
import urllib.request
from telebot import types
import requests
import os

In [3]:
# привязываем бота
bot = telebot.TeleBot(bot_token)

In [4]:
# функция детектирования лица на фото (выдаёт на выходе True/False)
def finder(tempimg):
    face_cascade = cv2.CascadeClassifier(path + r'/haarcascade_frontalface_default.xml')
    image = cv2.imread(tempimg)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5, minSize = (50,50))
    x = False
    if len(faces) > 0:
        x = True
    return x


In [5]:
# функция создания базы данных, если она ещё не была создана
# состоит из трёх таблиц
# Users - хранит идентификаторы пользователей по id чата из telegram
# Photos и Audios хранят пути к сохранённым фото/аудио файлам по идентификаторам пользователей
def db_start (path=path):
    conn = sqlite3.connect(path + r'/DSP_test_base.sqlite')
    cur = conn.cursor()
    cur.execute('PRAGMA foreign_keys;')
    
    cur.executescript('''
    CREATE TABLE IF NOT EXISTS Users (
        id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        uid    INTEGER UNIQUE
    );

    CREATE TABLE IF NOT EXISTS Audios (
        id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        user_id  INTEGER,
        path   TEXT UNIQUE,
        FOREIGN KEY (user_id) REFERENCES Users(id)
    );

    CREATE TABLE IF NOT EXISTS Photos (
        id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        user_id  INTEGER,
        path   TEXT UNIQUE,
        FOREIGN KEY (user_id) REFERENCES Users(id)
    );
    ''')

    conn.commit()
    conn.close()

In [6]:
# функция выдающая полный адрес+имя очередного голосового сообщения и сохраняющая сведения о нём в базе данных
def add_audio(chatid):
    conn = sqlite3.connect(path + r'/DSP_test_base.sqlite')
    cur = conn.cursor()
    cur.execute('SELECT id FROM Users WHERE uid = ? ', (chatid,))
    result = cur.fetchall()
    if result == []:
        cur.execute('INSERT INTO Users (uid) VALUES ( ? )', (chatid,))
        conn.commit()
        cur.execute('SELECT id FROM Users WHERE uid = ? ', (chatid,))
        result = cur.fetchall()
    cur.execute('SELECT id FROM Audios WHERE user_id = ?', (result[0][0],))
    num = len(cur.fetchall())
    fullpath = path + '/' + str(result[0][0]) + '_audio_message_' + str(num) + '.wav'
    cur.execute('INSERT INTO Audios (user_id, path)  VALUES ( ? , ? )', (result[0][0], fullpath))
    conn.commit()  
    conn.close()
    return fullpath

In [7]:
# функция выдающая полный адрес+имя очередного изображения и сохраняющая сведения о нём в базе данных
def add_photo(chatid):
    conn = sqlite3.connect(path + r'/DSP_test_base.sqlite')
    cur = conn.cursor()
    cur.execute('SELECT id FROM Users WHERE uid = ? ', (chatid,))
    result = cur.fetchall()
    if result == []:
        cur.execute('INSERT INTO Users (uid) VALUES ( ? )', (chatid,))
        conn.commit()
        cur.execute('SELECT id FROM Users WHERE uid = ? ', (chatid,))
        result = cur.fetchall()
    cur.execute('SELECT id FROM Photos WHERE user_id = ?', (result[0][0],))
    num = len(cur.fetchall())
    fullpath = path + '/' + str(result[0][0]) + '_photo_message_' + str(num) + '.jpeg'
    cur.execute('INSERT INTO Photos (user_id, path)  VALUES ( ? , ? )', (result[0][0], fullpath))
    conn.commit()  
    conn.close()
    return fullpath

In [8]:
# создаём базу данных
db_start()

# создаём команды /start и /help
@bot.message_handler(commands=['start'])  
def start_command(message):
    user_id = message.from_user.id
    bot.send_message(  
        message.chat.id,  
        'Hello! You have started the conversation'  
    )
    
@bot.message_handler(commands=['help'])  
def help_command(message):  
    bot.send_message(  
        message.chat.id,  
        'Just send messages',  
    )
    
# реакция на текстовое сообщение   
@bot.message_handler(content_types=['text'])  
def text(message):  
    bot.send_message(  
        message.chat.id,  
        'You have sent the text message',  
    )

# обрабатываем присланные фото
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    chat_id = message.chat.id
    file_info = bot.get_file(message.photo[-1].file_id)
    temp_photo = bot.download_file(file_info.file_path)
    src = path + '/' + message.photo[-1].file_id
    with open(src, 'wb') as new_file:
            new_file.write(temp_photo)
    ver = finder(src)
    os.remove(src)
    if ver:
        src = add_photo(chat_id)
        with open(src, 'wb') as new_file:
            new_file.write(temp_photo)
        rep = 'The face was found. The photo has been saved ' + src 
        bot.reply_to(message, rep)
    else:
        bot.reply_to(message, 'No face. Not saved.')
        
# обрабатываем присланные голосовые сообщения
@bot.message_handler(content_types=['voice'])
def handle_audio(message):
    chat_id = message.chat.id
    file_info = bot.get_file(message.voice.file_id)
    temp_audio = bot.download_file(file_info.file_path)
    src = path + '/' + message.voice.file_id
    with open(src, 'wb') as new_file:
            new_file.write(temp_audio)
    conv = AudioSegment.from_ogg(src)
    conv = conv.set_frame_rate(16000)
    os.remove(src)
    src = add_audio(chat_id)
    conv.export(src, format = "wav")
    rep = 'The voice message has been saved ' + src 
    bot.reply_to(message, rep)
   
    
bot.polling(none_stop = True, interval = 0)


